In [1]:
import pandas as pd
import numpy as np
import megamerge

In [2]:
segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data=[
        ["H001", "L",   0, 100],
        ["H001", "L", 100, 200],
        ["H001", "L", 200, 300],
        ["H001", "L", 300, 400],

        ["H001", "R",   0, 100],
    ]
)

data = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to", "measure", "category"],
    data=[                             # overlaps lengths
        ["H001", "L", 50,  140,  1.0, "A"],  # 50  40   0  0    [50, 40,  0,  0]
        ["H001", "L", 140, 160,  2.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 160, 180,  3.0, "B"],  #  0  20   0  0    [ 0, 20,  0,  0]
        ["H001", "L", 180, 220,  4.0, "B"],  #  0  20  20  0    [ 0, 20, 20,  0]
        ["H001", "L", 220, 240,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 240, 260,  5.0, "C"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 260, 280,  6.0, "D"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 280, 300,  7.0, "E"],  #  0   0  20  0    [ 0,  0, 20,  0]
        ["H001", "L", 300, 320,  8.0, "F"],  #  0   0     20    [ 0,  0,  0, 20]

        ["H001", "R",  10,  80,  9.0, "G"],  #  0   0     20
        ["H001", "R",  80, 120, 10.0, "H"],  #  0   0     20
    ]
)

expected_result = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to",  "measure longest value",  "category longest value"],
    data=[
        ["H001", "L",   0, 100,  1.0,  "A"],
        ["H001", "L", 100, 200,  1.0,  "B"],
        ["H001", "L", 200, 300,  5.0,  "C"],
        ["H001", "L", 300, 400,  8.0,  "F"],

        ["H001", "R",   0, 100,  9.0,  "G"],
    ]
)

In [5]:
res = megamerge.merge_interval_index(
    segments.set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    data    .set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    0
)

In [6]:
list(res)

[(array([0], dtype=uint64), array([50.])),
 (array([0, 1, 2, 3], dtype=uint64), array([40., 20., 20., 20.])),
 (array([3, 4, 5, 6, 7], dtype=uint64), array([20., 20., 20., 20., 20.])),
 (array([8], dtype=uint64), array([20.]))]

In [27]:
# generate segmentation
big_segments = pd.DataFrame(
    columns=["road", "cwy", "slk_from", "slk_to"],
    data   =[
        ["H001", "L",   index*500, (index+1)*500] for index in range(int(3000 / 0.010))
    ]
)

big_data = pd.DataFrame(
    columns= ["road", "cwy", "slk_from", "slk_to", "measure"],
    data   = [
        ["H001", "L",   (index)*10, (index+1)*10, index] for index in range(int(3000 / 0.010))
    ]
)

In [ ]:
res = megamerge.merge_interval_index(
    big_segments.set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    big_data    .set_index(["road","cwy"]).loc[("H001", "L"),["slk_from", "slk_to"]].to_numpy(dtype="f8"),
    0
)
res_list = list(res)

In [28]:
result = []
for index, overlap in res_list:
    data = big_data["measure"].iloc[index]
    namask = ~data.isna()
    data = data[namask]
    overlap = overlap[namask]
    
    result.append(
        np.sum(data*overlap/np.sum(overlap))
    )
result

KeyboardInterrupt: 

In [25]:
big_segments.__len__()

300000

In [19]:
big_data.head(5)

,road,cwy,slk_from,slk_to,measure
0,H001,L,0,10,0
1,H001,L,10,20,1
2,H001,L,20,30,2
3,H001,L,30,40,3
4,H001,L,40,50,4


In [26]:
300000/3000000

0.1